BDLE 2022

date du document  :  30/09/2022

Sahli Oussama

# TP1 2022 Préparation de données ETU


Préparation de données


## Préparation du TP

Vérifier que des ressources de calcul sont allouées à votre notebook est connecté (cf RAM  de disque indiqués en haut à droite) . Sinon cliquer sur le bouton connecter pour obtenir des ressources.




Pour accéder directement aux fichiers stockées sur votre google drive. Renseigner le code d'authentification lorsqu'il est demandé

Ajuster le nom de votre dossier : MyDrive/ens/bdle/DM1

In [6]:
# import os
# from google.colab import drive
# drive.mount("/content/drive")

# drive_dir = "/content/drive/MyDrive/ens/bdle/TP1"
# os.makedirs(drive_dir, exist_ok=True)
# os.listdir(drive_dir)

Installer pyspark et findspark :


In [3]:
!pip install -q pyspark
!pip install -q findspark

     |████████████████████████████████| 281.3 MB 60 kB/s 
     |████████████████████████████████| 199 kB 80.7 MB/s 


Démarrer la session spark

In [4]:
import os

# !find /usr/local/lib -name "pyspark"
os.environ["SPARK_HOME"] = "/usr/local/lib/python3.7/dist-packages/pyspark"
os.environ["JAVA_HOME"] = "/usr"

In [5]:
# Principaux import
import findspark
from pyspark.sql import SparkSession 
from pyspark import SparkConf  

# pour les dataframe et udf
from pyspark.sql import *  
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import *

# pour le chronomètre
import time

# initialise les variables d'environnement pour spark
findspark.init()

# Démarrage session spark 
# --------------------------
def demarrer_spark():
  local = "local[*]"
  appName = "TP"
  configLocale = SparkConf().setAppName(appName).setMaster(local).\
  set("spark.executor.memory", "6G").\
  set("spark.driver.memory","6G").\
  set("spark.sql.catalogImplementation","in-memory")
  
  spark = SparkSession.builder.config(conf = configLocale).getOrCreate()
  sc = spark.sparkContext
  sc.setLogLevel("ERROR")
  
  spark.conf.set("spark.sql.autoBroadcastJoinThreshold","-1")

  # On ajuste l'environnement d'exécution des requêtes à la taille du cluster (4 coeurs)
  spark.conf.set("spark.sql.shuffle.partitions","4")    
  print("session démarrée, son id est ", sc.applicationId)
  return spark
spark = demarrer_spark()

session démarrée, son id est  local-1665311604870


In [7]:
# on utilise 8 partitions au lieu de 200 par défaut
spark.conf.set("spark.sql.shuffle.partitions", "8")
print("Nombre de partitions utilisées : ", spark.conf.get("spark.sql.shuffle.partitions"))

Nombre de partitions utilisées :  8


In [8]:
# Optionnel :
# pour l'accès à spark UI : voir https://www.analyticsvidhya.com/blog/2020/11/a-must-read-guide-on-how-to-work-with-pyspark-on-google-colab-for-data-scientists/
# !wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
# !unzip ngrok-stable-linux-amd64.zip
# get_ipython().system_raw('./ngrok http 4050 &')
# !curl -s http://localhost:4040/api/tunnels

Redéfinir la fonction **display** pour afficher le resutltat des requêtes dans un tableau

In [9]:
import pandas as pd
from google.colab import data_table

# alternatives to Databricks display function.

def display(df, n=100):
  return data_table.DataTable(df.limit(n).toPandas(), include_index=False, num_rows_per_page=10)

def display2(df, n=20):
  pd.set_option('max_columns', None)
  pd.set_option('max_colwidth', None)
  return df.limit(n).toPandas()


Définir le tag **%%sql** pour pouvoir écrire plus simplement des requêtes en SQL dans une cellule

In [10]:
from IPython.core.magic import (register_line_magic, register_cell_magic, register_line_cell_magic)

def removeComments(query):
  result = ""
  for line in query.split('\n'):
    if not(line.strip().startswith("--")):
      result += line + "\n"
  return result

@register_line_cell_magic
def sql(line, cell=None):
    "To run a sql query. Use:  %%sql"
    val = cell if cell is not None else line
    tabRequetes = removeComments(val).split(";")
    derniere = None
    est_requete = False
    for r in tabRequetes:
        r = r.strip()
        if len(r) > 2:
          derniere = spark.sql(r)
          est_requete = r.lower().startswith('select')
    if(est_requete):
      return display(derniere)
    else:
      return print('ok')

In [11]:
# facultatif (à ne pas utiliser)
# %load_ext google.colab.data_table
# %unload_ext google.colab.data_table

## Accès aux données

### URL pour l'accès aux datasets

In [12]:
# URL du dossier PUBLIC_DATASET contenant des fichiers de données pour les TP
# ---------------------------------------------------------------------------
# en cas de problème avec le téléchargement des datasets, aller directement sur l'URL ci-dessous
PUBLIC_DATASET_URL = "https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4" 
PUBLIC_DATASET=PUBLIC_DATASET_URL + "/download?path="

print("URL du dossier contenant les datasets ", PUBLIC_DATASET_URL)

URL du dossier contenant les datasets  https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4


### Données de mobilité

Données issues du dataset YFCC

In [13]:
local_dir = "/local/data"
os.makedirs(local_dir, exist_ok=True)
os.listdir(local_dir)

[]

In [14]:
from urllib import request

# download dataset if not already donwloaded
def download_file(web_dir, local_dir, file):
  local_file = local_dir + "/" + file
  web_file = web_dir + "/" + file
  if(os.path.isfile(local_file)):
    print(file, "is already stored")
  else:
    print("downloading from URL: ", web_file , "save in : " + local_file)
    request.urlretrieve(web_file , local_file)

# user visits
web_dir = PUBLIC_DATASET + "YFCC_POI_dataset_K_H_LIM/dataset_IJCAI_2015/data-ijcai15/userVisits-ijcai15"
download_file(web_dir, local_dir, "userVisits-Toro.csv")

# poi 
web_dir = PUBLIC_DATASET + "YFCC_POI_dataset_K_H_LIM/dataset_IJCAI_2015/data-ijcai15/poiList-ijcai15"
download_file(web_dir, local_dir, "POI-Toro.csv")



os.listdir(local_dir)

downloading from URL:  https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4/download?path=YFCC_POI_dataset_K_H_LIM/dataset_IJCAI_2015/data-ijcai15/userVisits-ijcai15/userVisits-Toro.csv save in : /local/data/userVisits-Toro.csv
downloading from URL:  https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4/download?path=YFCC_POI_dataset_K_H_LIM/dataset_IJCAI_2015/data-ijcai15/poiList-ijcai15/POI-Toro.csv save in : /local/data/POI-Toro.csv


['userVisits-Toro.csv', 'POI-Toro.csv']

### Les visites

Lire les 2 premières lignes du  fichier csv en python. 
Est ce que le fichier a une ligne d'entête ?
Quel caractère délimite deux valeurs consécutives dans une ligne de données ?

In [15]:
f = open(local_dir + "/" + "userVisits-Toro.csv", "r")
print(f.readline()); print(f.readline())

"photoID";"userID";"dateTaken";"poiID";"poiTheme";"poiFreq";"seqID"

7941504100;"10007579@N00";1346844688;30;"Structure";1538;1



Lire le fichier des visites *sans* préciser le type des attributs.
Par défaut, tous les attributs sont considérés comme étant de type string.

In [16]:
user_visits = spark.read.option("header", "True").option("delimiter", ";").format("csv").load(local_dir + "/" + "userVisits-Toro.csv")
user_visits.show(3)
user_visits.printSchema()

+----------+------------+----------+-----+---------+-------+-----+
|   photoID|      userID| dateTaken|poiID| poiTheme|poiFreq|seqID|
+----------+------------+----------+-----+---------+-------+-----+
|7941504100|10007579@N00|1346844688|   30|Structure|   1538|    1|
|4886005532|10012675@N05|1142731848|    6| Cultural|    986|    2|
|4886006468|10012675@N05|1142732248|    6| Cultural|    986|    2|
+----------+------------+----------+-----+---------+-------+-----+
only showing top 3 rows

root
 |-- photoID: string (nullable = true)
 |-- userID: string (nullable = true)
 |-- dateTaken: string (nullable = true)
 |-- poiID: string (nullable = true)
 |-- poiTheme: string (nullable = true)
 |-- poiFreq: string (nullable = true)
 |-- seqID: string (nullable = true)



Lire le fichier en précisant le schéma : nom et type des attributs

In [17]:
schema = "photoID long, userID String, date Long, poiID int, poiTheme String, poiFreq int, seqID int"

user_visits = spark.read.option("header", "True").option("delimiter", ";").csv(local_dir + "/" + "userVisits-Toro.csv", schema = schema)
user_visits.persist()
user_visits.createOrReplaceTempView("user_visits")
user_visits.show(10)
user_visits.printSchema()

+----------+------------+----------+-----+---------+-------+-----+
|   photoID|      userID|      date|poiID| poiTheme|poiFreq|seqID|
+----------+------------+----------+-----+---------+-------+-----+
|7941504100|10007579@N00|1346844688|   30|Structure|   1538|    1|
|4886005532|10012675@N05|1142731848|    6| Cultural|    986|    2|
|4886006468|10012675@N05|1142732248|    6| Cultural|    986|    2|
|4885404441|10012675@N05|1142732373|    6| Cultural|    986|    2|
|4886008334|10012675@N05|1142732445|    6| Cultural|    986|    2|
|4886009150|10012675@N05|1142916492|    6| Cultural|    986|    3|
|7054481539|10012675@N05|1319327174|   13| Cultural|    964|    4|
|6908387594|10012675@N05|1319328255|   13| Cultural|    964|    4|
|6908381912|10012675@N05|1319331463|   13| Cultural|    964|    4|
|6908398496|10012675@N05|1319331886|   13| Cultural|    964|    4|
+----------+------------+----------+-----+---------+-------+-----+
only showing top 10 rows

root
 |-- photoID: long (nullable = 

Vérifier qu'il n'y a pas de date nulle

In [18]:
%%sql
select count(1) as nb_dates_nulles
from user_visits
where date is null

-- autre solution avec "case"
--select count(case when isnull(date) then date end) as nb_date_nulles
--from user_visits

,nb_dates_nulles
0,0



Vérifier qu'il n'y a aucune séquence associée à plusieurs utilisateurs

In [19]:
%%sql
select seqID
from user_visits
group by seqID
having count(distinct userID) > 1

,seqID



#### POI_sequence

La table **POI_sequence**(seqID, poiID). Les POI visités durant une séquence.
Pour simplifier on compte chaque POI une seule fois par séquence et on ne considère pas l'ordre de visite des POI.

Rmq: il n'est pas nécessaire de préciser le userID pour identifier une séquence.

In [20]:
%%sql
create or replace temp view POI_sequence as 
select distinct seqID, poiID
from user_visits;

select * from POI_sequence

,seqID,poiID
0,9,24
1,16,24
2,26,22
3,29,25
4,54,25
...,...,...
95,624,23
96,634,22
97,638,4
98,641,30


### Les lieux visités : POI
Ils sont appelés *Point Of Interest*

In [21]:
poi_schema = "poiID long, poiName String, latitude double, longitude double, theme String"

poi = spark.read.option("header", "True").option("delimiter", ";").csv(local_dir + "/" + "POI-Toro.csv", schema = poi_schema)
poi.show(3)
poi.printSchema()
poi.createOrReplaceTempView("POI")

+-----+------------------+--------+---------+-----+
|poiID|           poiName|latitude|longitude|theme|
+-----+------------------+--------+---------+-----+
|    1| Air_Canada_Centre|43.64333|-79.37917|Sport|
|    2|         BMO_Field|43.63278|-79.41861|Sport|
|    3|Maple_Leaf_Gardens|43.66222|-79.38028|Sport|
+-----+------------------+--------+---------+-----+
only showing top 3 rows

root
 |-- poiID: long (nullable = true)
 |-- poiName: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- theme: string (nullable = true)



In [22]:
%%sql
cache table POI;

SELECT * 
FROM POI

,poiID,poiName,latitude,longitude,theme
0,1,Air_Canada_Centre,43.643330,-79.379170,Sport
1,2,BMO_Field,43.632780,-79.418610,Sport
2,3,Maple_Leaf_Gardens,43.662220,-79.380280,Sport
3,4,Rogers_Centre,43.641390,-79.389170,Sport
4,5,Woodbine_Racetrack,43.712525,-79.602042,Sport
5,6,Art_Gallery_of_Ontario,43.653890,-79.392780,Cultural
6,7,Hockey_Hall_of_Fame,43.646976,-79.377253,Cultural
7,8,Ripley%27s_Aquarium_of_Canada,43.642481,-79.386050,Cultural
8,9,Ontario_Science_Centre,43.716670,-79.338330,Cultural
9,10,Riverdale_Farm,43.667111,-79.361294,Cultural


la liste des thèmes

In [23]:
%%sql
create or replace temp view themes as
select distinct theme
from POI
order by theme;

cache table themes;

select *
from themes

,theme
0,Amusement
1,Beach
2,Cultural
3,Shopping
4,Sport
5,Structure


## Exercice 1

#### 1) Les 10 POI les plus photographiés
indications: on peut compléter une requête SQL avec *LIMIT 10* pour borner la taille résultat (partiel) souhaité.

In [24]:
%%sql
select v.poiID, count(distinct(v.photoID)) as nbPhoto
from user_visits v
group by v.poiID
order by count(distinct(v.photoID)) desc
limit(10);


,poiID,nbPhoto
0,11,4139
1,22,3603
2,21,3591
3,16,3553
4,1,3506
5,4,3056
6,7,2053
7,23,1866
8,8,1736
9,25,1701


#### 2) Les visites avec date détaillée
Définir la table Visite_date(userID, seqID, poiID, date, annee, mois, jour, heure) contenant les visites avec la date détaillée composée des attributs : année, mois, jour, heure.
Indication : la date est initialement au format "unix". Voir la fonction de conversion de date from_unixtime et la fonction extract.

In [25]:
%%sql
create or replace temp view Visite_Date as

select userID, seqID, poiID, FROM_UNIXTIME(date) as date, 
EXTRACT(YEAR FROM FROM_UNIXTIME(date)) as annee, EXTRACT(MONTH FROM FROM_UNIXTIME(date)) as mois,
EXTRACT(DAY FROM FROM_UNIXTIME(date)) as jour, EXTRACT(HOUR FROM FROM_UNIXTIME(date)) as heure,
EXTRACT(MINUTE FROM FROM_UNIXTIME(date)) as minute, 
CAST ( REGEXP_EXTRACT( EXTRACT(SECOND FROM FROM_UNIXTIME(date)) , "^([0-9]+)\..+", 1)  as Int) as seconde

from user_visits;

select * from Visite_Date

,userID,seqID,poiID,date,annee,mois,jour,heure,minute,seconde
0,10007579@N00,1,30,2012-09-05 11:31:28,2012,9,5,11,31,28
1,10012675@N05,2,6,2006-03-19 01:30:48,2006,3,19,1,30,48
2,10012675@N05,2,6,2006-03-19 01:37:28,2006,3,19,1,37,28
3,10012675@N05,2,6,2006-03-19 01:39:33,2006,3,19,1,39,33
4,10012675@N05,2,6,2006-03-19 01:40:45,2006,3,19,1,40,45
...,...,...,...,...,...,...,...,...,...,...
95,10014440@N06,10,25,2008-07-09 08:53:33,2008,7,9,8,53,33
96,10014440@N06,10,25,2008-07-09 08:53:54,2008,7,9,8,53,54
97,10014440@N06,10,25,2008-07-09 08:54:10,2008,7,9,8,54,10
98,10014440@N06,10,25,2008-07-09 08:54:15,2008,7,9,8,54,15


#### 3a) Le nombre de POI par utilisateur
Le résultat (userID, nbPOI) est trié par nombre décroissant de POI.

In [26]:
%%sql
select userID, count(distinct(poiID)) as nbPOI
from user_visits 
group by(userID)
order by count(distinct(poiID)) desc;

,userID,nbPOI
0,34211328@N00,24
1,43139087@N00,23
2,84987970@N00,22
3,30624156@N00,20
4,63677124@N07,19
...,...,...
95,16048448@N00,8
96,31349854@N00,8
97,56020405@N08,8
98,36101699447@N01,8


#### 3b) Le nombre de POI par séquence
Définir la table Seq3plus(seqID, nbPoi) correspondant aux séquences qui contiennent au moins 3 POI distincts.
Afficher le résultat trié par nombre de POI décroissant puis numéro de séquence croissant.


In [27]:
%%sql
create or replace temp view Seq3plus as 
select seqID, count(distinct(poiID)) as nbPOI
from user_visits
group by seqID
having count(distinct(poiID)) >=3
order by nbPOI desc, seqID asc;

select * from seq3plus
order by nbPOI desc, seqID;

,seqID,nbPOI
0,298,13
1,4961,10
2,4351,9
3,5964,9
4,510,8
...,...,...
95,2377,4
96,2466,4
97,2472,4
98,2896,4


#### 4) Trajectoire



Définir la table Trajectoire(userID, seqID, listPOI).

listPOI est la liste des POI visités, pendant la séquence seqID, dans l'ordre chronologique

Indications:
pour la sequence 687 la listPOI est [7,16,4,8,4,16]

In [28]:
def trierPOI(t):
  result=[]
  for n in sorted(t):
    if(len(result)!=0) :
        if( result[len(result)-1] != n[1]) :
          result.append(n[1])
    else:
        result.append(n[1])
  return result

spark.udf.register("trierPOI", trierPOI, ArrayType(IntegerType()))

# test local
print(trierPOI([(13,0),(10,2), (12,1), (14,2),(11,2)]))

[2, 1, 0, 2]


In [29]:
%%sql
create or replace temp view Trajectoire as

select userID, seqID, trierPOI(collect_list((date,poiID))) as listePOI
from user_visits
group by userID, seqID;

cache table Trajectoire;

select * from Trajectoire
where size(listePOI) == 6
order by seqID

,userID,seqID,listePOI
0,14617133@N05,384,"[22, 28, 22, 28, 22, 28]"
1,20456447@N03,687,"[7, 16, 4, 8, 4, 16]"
2,20741443@N00,729,"[23, 21, 22, 7, 28, 23]"
3,27168489@N00,1358,"[16, 8, 28, 22, 23, 21]"
4,29352917@N00,1524,"[29, 30, 16, 29, 16, 29]"
5,30624156@N00,1598,"[28, 7, 30, 28, 22, 27]"
6,32827327@N03,1786,"[20, 6, 25, 11, 30, 24]"
7,33547369@N00,1877,"[23, 21, 23, 22, 23, 22]"
8,34128007@N04,1912,"[8, 16, 4, 1, 29, 16]"
9,34314322@N00,1993,"[30, 22, 28, 29, 30, 16]"


verification 

In [ ]:
 %%sql
 select *
 from user_Visits
 where seqID = 687
 order by date

,photoID,userID,date,poiID,poiTheme,poiFreq,seqID
0,7252786584,20456447@N03,1337381658,7,Cultural,2064,687
1,7252796922,20456447@N03,1337381696,7,Cultural,2064,687
2,7259485768,20456447@N03,1337381722,7,Cultural,2064,687
3,7259486332,20456447@N03,1337381807,7,Cultural,2064,687
4,7252977842,20456447@N03,1337381855,7,Cultural,2064,687
...,...,...,...,...,...,...,...
83,7256713412,20456447@N03,1337422407,16,Amusement,3553,687
84,7252808214,20456447@N03,1337422442,16,Amusement,3553,687
85,7252817310,20456447@N03,1337422552,16,Amusement,3553,687
86,7256727132,20456447@N03,1337422611,16,Amusement,3553,687


#### 5) Transitions
5a) Définir la table Transitions(poi1, poi2, nbTrans) avec nbTrans étant le nombre de déplacements directs de poi1 à poi2.
Indication: deux POI apparaissant successivement (poi1 suivi de poi2) dans une séquence forment un déplacement direct entre poi1 et poi2.
Une séquence contenant un seul POI ne correspond à aucun déplacement.

In [30]:
def transition(listePOI):
  result=[]
  if(len(listePOI)==1):
    return None
  for i in range (0, len(listePOI)-1):
    if(listePOI[i] != listePOI[i+1]):
      result.append((listePOI[i],listePOI[i+1]))
  return result

spark.udf.register("transition", transition, ArrayType( StructType( [StructField("p1" , IntegerType()), StructField("p2" , IntegerType()) ] ) ) )

# test local
print(transition([2,6,1,6]))

[(2, 6), (6, 1), (1, 6)]


In [31]:
%%sql
create or replace temp view Transition1 as 
select seqID, explode(transition(collect_list(poiID))) as transition
from user_visits
group by seqID;


select t.seqID, t.transition.p1,  t.transition.p2
from Transition1 t

,seqID,p1,p2
0,8,23,24
1,33,21,25
2,58,7,11
3,58,11,27
4,59,11,27
...,...,...,...
95,315,16,8
96,315,8,16
97,316,21,27
98,316,27,11


In [32]:
%%sql
create or replace temp view Transition2 as 
select t.transition.p1,  t.transition.p2, count(*) as nbtransition
from Transition1 t
group by t.transition.p1,  t.transition.p2;

select * from Transition2
order by nbtransition desc;

,p1,p2,nbtransition
0,23,21,82
1,22,28,70
2,28,22,64
3,21,23,63
4,7,30,57
...,...,...,...
95,7,11,4
96,23,29,4
97,11,25,4
98,19,15,4


b) Definir la vue 
Transition_relative avec des valeurs de transition relatives au nombre total de transition partant d'un POI.

Indication: créer une vue donnant le nombre total de transitions par poi.

In [33]:
%%sql
create or replace temp view Total_transition as 
select p1, sum(nbtransition) as total
from Transition2
group by p1
order by p1 asc;


select * from Total_transition

,p1,total
0,1,45
1,2,18
2,3,24
3,4,61
4,6,61
5,7,153
6,8,93
7,9,1
8,10,4
9,11,68


In [34]:
%%sql
create or replace temp view Transition_relative as 
select  t.p1, t2.p2, t2.nbtransition/t.total as rel
from Transition2 t2,Total_transition t
where t2.p1=t.p1 ;


select * from Transition_relative
order by  p1

,p1,p2,rel
0,1,16,0.088889
1,1,19,0.022222
2,1,7,0.111111
3,1,8,0.155556
4,1,24,0.044444
...,...,...,...
95,11,29,0.117647
96,13,16,0.200000
97,13,21,0.100000
98,13,4,0.050000


#### Distances entre POIs
table Distance(poi1, poi2, distance)

In [35]:
import math
import cmath

In [36]:
# les import doivent être fait dans la fonction udf

#retourne la distance en mètre
def distance(a,b,c,d):
  #on convertit les angles en radiant
  import math

  a=a* (math.pi / 180)
  b=b * (math.pi / 180)
  c=c * (math.pi / 180)
  d=d * (math.pi / 180) 
  R=6367445  # rayon de la terre en mètre
  return R*math.acos(math.sin(a)*math.sin(b) + (math.cos(a)*math.cos(b)*math.cos(c-d)) ) 

spark.udf.register("distance", distance, DoubleType() ) # il faut faire attention à bien choisir le type ici, sinon la valeur retourné est null

print( distance(2.350,3.063,48.856,50.63) )

212272.1385712766


In [37]:
%%sql
select p1.poiID,p2.poiID,distance(p1.latitude,p2.latitude,p1.longitude,p2.longitude) as distance
from POI p1 full join POI p2

,poiID,poiID,distance
0,1,1,0.094882
1,1,2,3381.838674
2,1,3,2101.196600
3,1,4,832.623694
4,1,5,19494.102274
...,...,...,...
95,4,6,1419.169639
96,4,7,1141.860741
97,4,8,278.678335
98,4,9,9310.654348


### 6) DuréeVisitePOI 
La durée moyenne de visite d'un POI.
Indication, pour une série d'événements consécutifs associés à un même POI dans une séquence, la durée de visite est la différence de date entre le 1er et le dernier événement.

In [38]:
%%sql
select poiID,avg(duree_visite) as duree_moyenne
from (select userID,seqID,poiID, max(date)-min(date) as duree_visite from user_visits group by userID,seqID,poiID) duree_visite
group by poiID;


,poiID,duree_moyenne
0,14,965.973684
1,12,3136.986486
2,13,2124.133333
3,18,1037.400000
4,23,1468.289225
5,6,1379.530435
6,16,2971.145068
7,9,3673.694444
8,17,6392.653846
9,10,2013.134146


## Exercice 2

Charger les données de Geonames

In [40]:
import zipfile

#geonames
web_dir = PUBLIC_DATASET + "geonames_ALL"

download_file(web_dir, local_dir, "allCountries.zip")

#unzip
local_file = local_dir + "/" + "allCountries.txt"
if(os.path.isfile(local_file)):
    print(local_file, "is already stored")
else:
  with zipfile.ZipFile(local_dir + "/" + "allCountries.zip", 'r') as zip_ref:
    zip_ref.extractall(local_dir)

os.listdir(local_dir)

downloading from URL:  https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4/download?path=geonames_ALL/allCountries.zip save in : /local/data/allCountries.zip


['allCountries.zip', 'userVisits-Toro.csv', 'POI-Toro.csv', 'allCountries.txt']

In [41]:
geonames =  spark.read.option("header", "False").option("delimiter", "\t").format("csv").load(local_dir + "/" + "allCountries.txt")
geonames.createOrReplaceTempView("geonames")
geonames.show(3)

+-------+--------------------+--------------------+--------------------+--------+-------+---+---+---+-----+----+----+----+----+----+----+----+--------------+----------+
|    _c0|                 _c1|                 _c2|                 _c3|     _c4|    _c5|_c6|_c7|_c8|  _c9|_c10|_c11|_c12|_c13|_c14|_c15|_c16|          _c17|      _c18|
+-------+--------------------+--------------------+--------------------+--------+-------+---+---+---+-----+----+----+----+----+----+----+----+--------------+----------+
|2986043|  Pic de Font Blanca|  Pic de Font Blanca|Pic de Font Blanc...|42.64991|1.53335|  T| PK| AD| null|  00|null|null|null|   0|null|2860|Europe/Andorra|2014-11-05|
|2994701|            Roc Mélé|            Roc Mele|Roc Mele,Roc Mele...|42.58765|1.74028|  T| MT| AD|AD,FR|  00|null|null|null|   0|null|2803|Europe/Andorra|2014-11-05|
|3007683|Pic des Langounelles|Pic des Langounelles|Pic des Langounelles|42.61203|1.47364|  T| PK| AD|AD,FR|  00|null|null|null|   0|null|2685|Europe/Andorr

In [42]:
geonames.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: string (nullable = true)
 |-- _c11: string (nullable = true)
 |-- _c12: string (nullable = true)
 |-- _c13: string (nullable = true)
 |-- _c14: string (nullable = true)
 |-- _c15: string (nullable = true)
 |-- _c16: string (nullable = true)
 |-- _c17: string (nullable = true)
 |-- _c18: string (nullable = true)



### 1) Geonames


#####1a) schéma de Geonames

Définir Geonames2 en précisant le schéma : nom et type des attributs.
Se limiter au 9 premiers attributs donnant des informations sur l'identifiant d'un POI, ses noms, sa position GPS, le code du pays..., ), d'après le document [readme.txt](https://nuage.lip6.fr/s/kaBX3sF3YEzMZHK) dans le dossier [geonames_ALL](https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4?path=%2Fgeonames_ALL)

In [43]:
%%sql
create or replace temp view Geonames2 as 
select cast(_c0 as long) as geonameID, 
cast(_c1 as varchar(200)) as name,
cast(_c3 as varchar(10000)) as alternate_name,
cast(_c4 as double) as latitude,
cast(_c5 as double) as longitude,
cast(_c6 as char(1)) as feature_class,
cast(_c7 as varchar(10)) as feature_code,
cast(_c8 as char(2)) as country_code

from Geonames;

select * from Geonames2

,geonameID,name,alternate_name,latitude,longitude,feature_class,feature_code,country_code
0,2986043,Pic de Font Blanca,"Pic de Font Blanca,Pic du Port",42.64991,1.53335,T,PK,AD
1,2994701,Roc Mélé,"Roc Mele,Roc Meler,Roc Mélé",42.58765,1.74028,T,MT,AD
2,3007683,Pic des Langounelles,Pic des Langounelles,42.61203,1.47364,T,PK,AD
3,3017832,Pic de les Abelletes,"Pic de la Font-Negre,Pic de la Font-Nègre,Pic ...",42.52535,1.73343,T,PK,AD
4,3017833,Estany de les Abelletes,"Estany de les Abelletes,Etang de Font-Negre,Ét...",42.52915,1.73362,H,LK,AD
...,...,...,...,...,...,...,...,...
95,3038902,Tosquers,None,42.56587,1.48763,T,SLP,AD
96,3038903,Canal del Tosquer,None,42.57114,1.52067,H,STM,AD
97,3038904,Bosc del Tosquer,None,42.57189,1.52369,V,FRST,AD
98,3038905,Bosc del Tosquer,None,42.53793,1.60070,V,FRST,AD


##### 1b) Extrait pour le Canada

Définir la vue Geonames_canada pour les POI situés au Canada.

In [44]:
%%sql
create or replace temp view Geonames_canada as
select geonameID, name, alternate_name, latitude, longitude, feature_class, feature_code from Geonames2
where country_code = 'CA';

cache table Geonames_canada;

select * from  Geonames_canada;

,geonameID,name,alternate_name,latitude,longitude,feature_class,feature_code
0,3424953,Virgin Rocks,None,46.42886,-50.81995,U,RFU
1,3425041,Eastern Shoals,None,46.44743,-50.48577,U,SHSU
2,3426061,Downing Basin,None,47.05216,-50.79163,U,TRGU
3,3831549,Kane Basin,"Bassin Kane,Kane Basin",79.38860,-70.42541,H,STRT
4,3831631,Georges Shoal,"Georges Bank,St. George's Bank,St. George's Shoal",41.65444,-67.73242,U,SHLU
...,...,...,...,...,...,...,...
95,5881729,Aalders Landing,None,44.83345,-64.91549,L,AREA
96,5881730,Aalders Lang Brook,None,44.86685,-64.29875,H,STM
97,5881731,Aalders Lang Meadow,None,44.85015,-64.31545,T,PLN
98,5881732,Aaltanhash Inlet,Aaltenash Bight,53.12854,-128.51789,H,INLT


### 2) Association entre les POI et Geonames
Compléter des POI avec des attributs de geonames (par exemple name et [feature code](http://www.geonames.org/export/codes.html))

In [45]:
%%sql
select p.poiID, p.poiName, p.latitude, p.longitude, p.theme, g.name, g.feature_code
from  POI p left outer join Geonames2 g on  p.poiID=g.geonameID

,poiID,poiName,latitude,longitude,theme,name,feature_code
0,2,BMO_Field,43.632780,-79.418610,Sport,None,None
1,12,Toronto_Zoo,43.820278,-79.182778,Cultural,Takht-e Qeyşar,PPL
2,26,Yorkdale_Shopping_Centre,43.725599,-79.452696,Shopping,Sheykh ‘Ajam,PPL
3,28,Old_City_Hall_(Toronto),43.652500,-79.381670,Structure,None,None
4,29,Ontario_Legislative_Building,43.662447,-79.391708,Structure,Sharū,PPL
5,30,Union_Station_(Toronto),43.645280,-79.380560,Structure,Shāreh Kanī,PPL
6,9,Ontario_Science_Centre,43.716670,-79.338330,Cultural,Kūh-e Yalmī,MT
7,11,Royal_Ontario_Museum,43.667500,-79.394170,Cultural,Takht-e Jān,STM
8,22,Nathan_Phillips_Square,43.652500,-79.383610,Beach,Shahīd Madanī,PPL
9,14,Canadian_National_Exhibition,43.633330,-79.416670,Amusement,Kūh-e Takht-e Arreh,HLL
